In [10]:
import pandas as pd
from textblob import TextBlob
import nltk

In [11]:
df = pd.read_pickle('data/re_merge/clean.pickle')
df.reset_index(inplace=True)
df.head(3)

,index,nyt_id,org,hl,lead,date_pub
0,3,4fc0a09745c1498b0d3ba216,none,MARIJUANA SMOKING IS REPORTED SAFE Hemp Leaves...,PANAMA Nov 14 A Panaman Judge recently senten...,1926-11-21
1,9,4fc1d8e345c1498b0d4ccb9f,none,USE OF MARIJUANA SPREADING IN WEST Poisonous W...,DENVER Sept 13 Although as appalling in its e...,1934-09-16
2,12,4fc1ebab45c1498b0d528e5b,The Associated Press,RHODE ISLAND TO END WEED AS DRUG SOURCE State ...,PROVIDENCE RI Jan 19 Rhode Island authorities ...,1935-01-20


In [12]:
li = df[['lead']].head().values.tolist()
li

[['PANAMA Nov 14  A Panaman Judge recently sentenced an American seaman Hamilton Main to a year of penal confinement for smoking and having in his possession cigarettes made of the leaves of the cannabis indica known also as marijuana canjac and by various other names and often incorrectly referred to as hashish'],
 ['DENVER Sept 13  Although as appalling in its effects on the human mind and body as narcotics the consumption of marijuana appears to be proceeding virtually unchecked in Colorado and other Western States with a large SpanishAmerican population'],
 ['PROVIDENCE RI Jan 19 Rhode Island authorities are planning a Spring drive to eradicate the marijuana or Mexico weed which long has been the source of large supplies of the dangerous narcotic drug known as hashish'],
 ['To enable policemen to familiarize themselves with the appearance of marijuana pots of the narcotic weed have been placed on exhibition during the past few days in the assembly rooms of station houses in various

In [13]:
# stem words and tokenize

stemmer = nltk.stem.porter.PorterStemmer()
li_stem = []
for article in li:
    article_temp = []
    for string in article:
        for word in TextBlob(string).words:
            article_temp.append(stemmer.stem(word))
    li_stem.append(article_temp)
li_stem[0]

[u'PANAMA',
 u'Nov',
 u'14',
 u'A',
 u'Panaman',
 u'Judg',
 u'recent',
 u'sentenc',
 u'an',
 u'American',
 u'seaman',
 u'Hamilton',
 u'Main',
 u'to',
 u'a',
 u'year',
 u'of',
 u'penal',
 u'confin',
 u'for',
 u'smoke',
 u'and',
 u'have',
 u'in',
 u'hi',
 u'possess',
 u'cigarett',
 u'made',
 u'of',
 u'the',
 u'leav',
 u'of',
 u'the',
 u'cannabi',
 u'indica',
 u'known',
 u'also',
 u'as',
 u'marijuana',
 u'canjac',
 u'and',
 u'by',
 u'variou',
 u'other',
 u'name',
 u'and',
 u'often',
 u'incorrectli',
 u'refer',
 u'to',
 u'as',
 u'hashish']

In [14]:
# remove stopwords
from nltk.corpus import stopwords
li_sw = []
for article in li_stem:
    filtered_words = [word for word in article if word not in stopwords.words('english')]
    li_sw.append(filtered_words)
li_sw[0]

[u'PANAMA',
 u'Nov',
 u'14',
 u'A',
 u'Panaman',
 u'Judg',
 u'recent',
 u'sentenc',
 u'American',
 u'seaman',
 u'Hamilton',
 u'Main',
 u'year',
 u'penal',
 u'confin',
 u'smoke',
 u'hi',
 u'possess',
 u'cigarett',
 u'made',
 u'leav',
 u'cannabi',
 u'indica',
 u'known',
 u'also',
 u'marijuana',
 u'canjac',
 u'variou',
 u'name',
 u'often',
 u'incorrectli',
 u'refer',
 u'hashish']

In [32]:
# join words
documents = []
for article in li_sw:
    documents.append(' '.join(article))
documents = [str(x) for x in documents]
documents

['PANAMA Nov 14 A Panaman Judg recent sentenc American seaman Hamilton Main year penal confin smoke hi possess cigarett made leav cannabi indica known also marijuana canjac variou name often incorrectli refer hashish',
 'DENVER Sept 13 Although appal effect human mind bodi narcot consumpt marijuana appear proceed virtual uncheck Colorado Western State larg SpanishAmerican popul',
 'PROVID RI Jan 19 Rhode Island author plan Spring drive erad marijuana Mexico weed long ha sourc larg suppli danger narcot drug known hashish',
 'To enabl policemen familiar themselv appear marijuana pot narcot weed place exhibit dure past day assembl room station hous variou section Brooklyn',
 'Three million dollar worth bootleg price marijuana weed use cigarett form narcot wa burn vacant lot Brooklyn yesterday presenc Polic Commission Valentin Captain Joseph Mooney narcot squad polic offici']

In [33]:
# get bigrams
from nltk.util import ngrams
from collections import defaultdict
from operator import itemgetter
import copy
from collections import defaultdict

documents = copy.deepcopy(li_sw)

counter = defaultdict(int)

n = 2
for doc in documents:
    bigrams = ngrams(doc, n)
    for gram in bigrams:
        counter[gram] += 1

for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:30]:
    phrase = " ".join(gram)
    #print '%20s %i' % (phrase, count)

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

#CountVectorizer is a class; so `vectorizer` below represents an instance of that object.
vectorizer = CountVectorizer(ngram_range=(1,2))

# call `fit` to build the vocabulary
vectorizer.fit(documents)

# then, use `get_feature_names` to return the tokens
print vectorizer.get_feature_names()

# finally, call `transform` to convert text to a bag of words
x = vectorizer.transform(text)
type(x)

AttributeError: 'list' object has no attribute 'lower'

In [34]:
print 'Sparse Matrix'
print x # A compressed version; the "sparse" matrix.
print type(x) 
print
print 'Matrix'
x_back = x.toarray()
print type(x_back)
print x_back

Sparse Matrix
Three million dollar worth bootleg price marijuana weed use cigarett form narcot wa burn vacant lot Brooklyn yesterday presenc Polic Commission Valentin Captain Joseph Mooney narcot squad polic offici
<type 'unicode'>

Matrix


AttributeError: 'unicode' object has no attribute 'toarray'

In [31]:
pd.DataFrame(x_back, columns=vectorizer.get_feature_names())

ValueError: Shape of passed values is (235, 3), indices imply (221, 3)

In [25]:
#### TF: frequency in this document
#### IDF: inverse frequency in the corpus

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
doc_vectors = vectorizer.fit_transform(documents)

classes = np.array(['pos']*50 + ['neg']*50)


model = MultinomialNB().fit(doc_vectors, classes)

ValueError: Found arrays with inconsistent numbers of samples: [  5 100]